<a href="https://colab.research.google.com/github/hariharanrl/NLP/blob/master/NLP_Text_Classification_Vader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import os
os.chdir('/content/drive/My Drive')

In [2]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [0]:
data=pd.read_csv('task_b_distant.tsv',sep='\t')

In [4]:
data.head()

,id,text,average,std
0,1159533703758061570,@USER His ass need to stay up 😂😂,0.176201,0.267800
1,1159533712079503361,@USER Trump is a fucking idiot his dementia is...,0.173800,0.239891
2,1159533713044234241,@USER HELL YES! His grinned and thumbs up are ...,0.199833,0.212598
3,1159533718345830400,@USER Can't wait to see the shit show his deat...,0.144666,0.235204
4,1159533739871002625,@USER @USER @USER This guys is dumb check his ...,0.116019,0.186851


In [0]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()


In [13]:
sid.polarity_scores(data.loc[15]['text'])

{'compound': -0.802, 'neg': 0.31, 'neu': 0.69, 'pos': 0.0}

In [17]:
data['scores']=data['text'].apply(lambda text:sid.polarity_scores(text))
data.head()

,id,text,average,std,scores
0,1159533703758061570,@USER His ass need to stay up 😂😂,0.176201,0.267800,"{'neg': 0.333, 'neu': 0.667, 'pos': 0.0, 'comp..."
1,1159533712079503361,@USER Trump is a fucking idiot his dementia is...,0.173800,0.239891,"{'neg': 0.456, 'neu': 0.544, 'pos': 0.0, 'comp..."
2,1159533713044234241,@USER HELL YES! His grinned and thumbs up are ...,0.199833,0.212598,"{'neg': 0.447, 'neu': 0.288, 'pos': 0.265, 'co..."
3,1159533718345830400,@USER Can't wait to see the shit show his deat...,0.144666,0.235204,"{'neg': 0.218, 'neu': 0.559, 'pos': 0.223, 'co..."
4,1159533739871002625,@USER @USER @USER This guys is dumb check his ...,0.116019,0.186851,"{'neg': 0.355, 'neu': 0.645, 'pos': 0.0, 'comp..."


In [18]:
data['label']=data['scores'].apply(lambda d:d['compound'])
data.head()

,id,text,average,std,scores,label
0,1159533703758061570,@USER His ass need to stay up 😂😂,0.176201,0.267800,"{'neg': 0.333, 'neu': 0.667, 'pos': 0.0, 'comp...",-0.5423
1,1159533712079503361,@USER Trump is a fucking idiot his dementia is...,0.173800,0.239891,"{'neg': 0.456, 'neu': 0.544, 'pos': 0.0, 'comp...",-0.7713
2,1159533713044234241,@USER HELL YES! His grinned and thumbs up are ...,0.199833,0.212598,"{'neg': 0.447, 'neu': 0.288, 'pos': 0.265, 'co...",-0.6988
3,1159533718345830400,@USER Can't wait to see the shit show his deat...,0.144666,0.235204,"{'neg': 0.218, 'neu': 0.559, 'pos': 0.223, 'co...",0.0258
4,1159533739871002625,@USER @USER @USER This guys is dumb check his ...,0.116019,0.186851,"{'neg': 0.355, 'neu': 0.645, 'pos': 0.0, 'comp...",-0.7650


In [19]:
data['class']=data['label'].apply(lambda c:'not' if c>=0 else 'off')
data.head()

,id,text,average,std,scores,label,class
0,1159533703758061570,@USER His ass need to stay up 😂😂,0.176201,0.267800,"{'neg': 0.333, 'neu': 0.667, 'pos': 0.0, 'comp...",-0.5423,off
1,1159533712079503361,@USER Trump is a fucking idiot his dementia is...,0.173800,0.239891,"{'neg': 0.456, 'neu': 0.544, 'pos': 0.0, 'comp...",-0.7713,off
2,1159533713044234241,@USER HELL YES! His grinned and thumbs up are ...,0.199833,0.212598,"{'neg': 0.447, 'neu': 0.288, 'pos': 0.265, 'co...",-0.6988,off
3,1159533718345830400,@USER Can't wait to see the shit show his deat...,0.144666,0.235204,"{'neg': 0.218, 'neu': 0.559, 'pos': 0.223, 'co...",0.0258,not
4,1159533739871002625,@USER @USER @USER This guys is dumb check his ...,0.116019,0.186851,"{'neg': 0.355, 'neu': 0.645, 'pos': 0.0, 'comp...",-0.7650,off


In [20]:
data.isnull().sum()

id         0
text       0
average    0
std        0
scores     0
label      0
class      0
dtype: int64

In [0]:
blanks=[]
for i,j,text,avg,std in data.itertuples():
  if type(text)==str:
    if text.isspace():
      blanks.append(i)
print(len(blanks))

In [0]:
#data['label']="not"
#data.loc[data['average']>0.20,'label']="off"


In [22]:
data['class'].value_counts()

off    161360
not     27367
Name: class, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
X=data['text']
y=data['class']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

text_cls_nb=Pipeline([('tfidf',TfidfVectorizer()),('clf',MultinomialNB()),])
text_cls_svm=Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC()),])

In [25]:
text_cls_nb.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [0]:
prediction=text_cls_nb.predict(X_test)

In [27]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,prediction))

[[   14  5413]
 [    1 32318]]


In [28]:
print(metrics.classification_report(y_test,prediction))

              precision    recall  f1-score   support

         not       0.93      0.00      0.01      5427
         off       0.86      1.00      0.92     32319

    accuracy                           0.86     37746
   macro avg       0.89      0.50      0.46     37746
weighted avg       0.87      0.86      0.79     37746



In [29]:
print(metrics.accuracy_score(y_test,prediction))

0.8565675833200869


In [35]:
print(text_cls_nb.predict(['What the heck']))

['off']


In [32]:
sid.polarity_scores('Oh what i have heard is it true')

{'compound': 0.4215, 'neg': 0.0, 'neu': 0.682, 'pos': 0.318}